In [37]:
import pandas as pd
import numpy as np
from pandas import DataFrame
import math

In [38]:
df_Market_Cap = pd.read_pickle("UsableData/MarketCap.pkl")
df_Employees = pd.read_pickle("UsableData/Employees.pkl")
df_Book_Value = pd.read_pickle("UsableData/BookValue.pkl")
df_Returns = pd.read_pickle("UsableData/Returns.pkl").T

In [39]:
df_eligible = df_Market_Cap
for column in df_eligible:
    df_eligible[column].astype("bool")
    df_eligible[column] = False
df_eligible.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
Index: 1720 entries, 05P.D to ZZSG.DE^L07
Columns: 27 entries, 1995 to 2021
dtypes: bool(27)
memory usage: 58.8+ KB


In [40]:
def unusableReturns(ric, year):
    selection = df_Returns[ric][(df_Returns.index >= f"{year}-01-01") & (df_Returns.index <= f"{year}-12-31")]
    return bool(selection.isna().values.any())

In [41]:
for ric in df_Market_Cap.index:
    for year in df_Market_Cap.columns:
        mc = df_Market_Cap.loc[ric, year]
        emp = df_Employees.loc[ric, year]
        bv = df_Book_Value.loc[ric, year]
        if(math.isnan(mc) or math.isnan(emp) or math.isnan(bv) or unusableReturns(ric, year)):
            df_eligible.at[ric, year] = False
        else:
            df_eligible.at[ric, year] = True
df_eligible

Year,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
RIC,,,,,,,,,,,,,,,,,,,,,
05P.D,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
0955.HK^D12,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
0FIBG.DE^F10,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
0RN6.L^G19,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
123F.DE,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZO1G.H,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
ZOOG.BE,False,False,False,False,False,False,True,True,True,True,...,True,True,False,False,False,False,False,False,False,False
ZPFGn.MU,False,False,False,False,False,True,True,True,True,True,...,True,True,False,False,False,False,False,True,True,True


In [42]:
for i in range(len(df_eligible.columns)):
    df = df_eligible[df_eligible[df_eligible.columns[i]] == True]
    print(f"{df_eligible.columns[i]}: {len(df.index)}")

1995: 22
1996: 39
1997: 76
1998: 81
1999: 219
2000: 365
2001: 517
2002: 524
2003: 508
2004: 575
2005: 574
2006: 606
2007: 706
2008: 747
2009: 720
2010: 697
2011: 693
2012: 673
2013: 619
2014: 541
2015: 511
2016: 472
2017: 474
2018: 494
2019: 538
2020: 566
2021: 571


In [43]:
df_eligible.to_pickle("UsableData/EligibleStocks.pkl")